<a href="https://colab.research.google.com/github/souvikroy/Question-Quest-recommendation/blob/main/Quest_Reco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import networkx as nx
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df_topic_LU = pd.read_csv('/content/topic_LU connection.csv')
df_eq = pd.read_csv('/content/Existing_questions_output.csv')
df_nq = pd.read_csv('/content/New_questions_output.csv')
pub_usermetrics = pd.read_csv('/content/pub_usermetrics.csv')


In [ ]:
# API input
user_id = 6036
topic_name = "Associative property for operations of rational numbers"
no_recommendation = 20

In [ ]:
# unique topic name
df_topic_LU = df_topic_LU.dropna(subset=['topic_name'])
# Find unique combinations of 'topic_name' and 'lu_name'
unique_topic_lu = df_topic_LU[['topic_name', 'lu_name']].drop_duplicates()
unique_topic_lu

,topic_name,lu_name
0,Circle,Definition of an angle
12,Chance and probability,"Representation of fraction (1/2, 1/3, 1/4, 1/5..."
21,"Common factors, Common multiples, Co-primes",Definition of common multiples
41,"Common factors, Common multiples, Co-primes",Common factors of numbers
53,"Equivalent fractions, Simplest form of a fraction",Common factors of numbers
...,...,...
10606,Alive: moving and responding,Living things show movement and respond to sti...
10623,Alive: need food and energy,"Living things need food, energy through respir..."
10638,"Alive: grow, reproduce and die","Living things reproduce, grow, and have a defi..."
10660,Human Digestive System,Definition of holozoic nutrition


In [ ]:
# question level data
q_total1 = pd.concat([df_nq, df_eq], ignore_index=True)
q_total1['type'] = q_total1.apply(lambda row: 'new' if row.name in df_eq.index else 'repeat', axis=1)
q_total1 = q_total1.drop(columns=['competency_score_range', 'Predicted_competency_score_range'], errors='ignore')
q_total1 = pd.merge(q_total1, unique_topic_lu, left_on='learning_unit', right_on='lu_name', how='left')
if 'lu_name' in q_total1.columns:
    q_total1 = q_total1.drop(columns=['lu_name'])
q_total1

,question_text,Subject,grade,chapter,question_id,total_attempt,3PL,learning_unit,learning_objective,competency_score,Bloom_tag,type,topic_name
0,Complete given analogy by relating the type of...,Biology,6,Getting to know plants,2027,20,-2.944439,Plant's stem structure and functions,Recognize the standard symbol for an electric ...,0.950000,Remember,new,Stem
1,Complete given analogy by relating the type of...,Biology,6,Getting to know plants,2027,20,-2.944439,Plant's stem structure and functions,Recognize the standard symbol for an electric ...,0.950000,Remember,new,Roots
2,The image shows a habitat. Identify the biotic...,Biology,6,The Living Organisms — Characteristics and Hab...,5737,20,-2.944439,Definition of biotic components,Classify observed objects based on their state...,0.950000,Apply,new,Habitat and Adaptation
3,Complete the given analogy by relating the typ...,Biology,6,Getting to know plants,1941,19,-2.890372,Plant's stem structure and functions,Analyze the significance of conductors and ins...,0.947368,Apply,new,Stem
4,Complete the given analogy by relating the typ...,Biology,6,Getting to know plants,1941,19,-2.890372,Plant's stem structure and functions,Analyze the significance of conductors and ins...,0.947368,Apply,new,Roots
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18097,The diagram shows plant P and plant Q.Identify...,Biology,7,Reproduction in plants,7884,25,3.178054,Definition of reproduction in plants,Demonstrate the steps to maintain personal hyg...,0.040000,Apply,repeat,Asexual mode of reproduction in various organisms
18098,The diagram shows plant P and plant Q.Identify...,Biology,7,Reproduction in plants,7884,25,3.178054,Definition of reproduction in plants,Demonstrate the steps to maintain personal hyg...,0.040000,Apply,repeat,Modes of reproduction and asexual mode of repr...
18099,The diagram shows plant P and plant Q.Identify...,Biology,7,Reproduction in plants,7884,25,3.178054,Definition of reproduction in plants,Demonstrate the steps to maintain personal hyg...,0.040000,Apply,repeat,Asexual mode of reproduction in various organisms
18100,The diagram shows plant P and plant Q.Identify...,Biology,7,Reproduction in plants,7884,25,3.178054,Definition of reproduction in plants,Demonstrate the steps to maintain personal hyg...,0.040000,Apply,repeat,Modes of reproduction and asexual mode of repr...


In [ ]:
# new vs repeat criteria - 15 percentile
per_15_attempt = pub_usermetrics['attempt'].quantile(0.15)

pub_usermetrics['type'] = 'new'
pub_usermetrics.loc[pub_usermetrics['attempt'] >= per_15_attempt, 'type'] = 'repeat'
pub_usermetrics1 = pub_usermetrics[['user_id','CR', 'LI', 'LG', 'type']].copy()
pub_usermetrics1

,user_id,CR,LI,LG,type
0,2038,0.000000,0.000000,0.0,new
1,5746,40.814815,14.000000,393.0,new
2,5987,46.000000,13.000000,2880.0,repeat
3,6005,91.333333,0.000000,285.0,repeat
4,6021,64.000000,81.956522,95.0,new
...,...,...,...,...,...
9492,26302,27.000000,67.000000,0.0,new
9493,26305,100.000000,18.000000,43.0,new
9494,26306,100.000000,36.000000,31.0,new
9495,26307,100.000000,27.000000,37.0,new


In [ ]:
# user segment
def create_matrix_segments(df):
  """
  Creates matrix segments of users based on CR, LI, and LG.

  Args:
      df: A pandas DataFrame containing user data with 'CR', 'LI', 'LG', and 'user_id' columns.

  Returns:
      A pandas DataFrame with user segments.
  """

  # Calculate quantiles for CR, LI, and LG
  cr_quantiles = df['CR'].quantile([0.33, 0.66])
  li_quantiles = df['LI'].quantile([0.33, 0.66])
  lg_quantiles = df['LG'].quantile([0.33, 0.66])

  # Define segmentation logic based on quantiles
  def segment_user(row):
      cr_segment = 'LOW'
      if row['CR'] >= cr_quantiles[0.66]:
          cr_segment = 'HIGH'
      elif row['CR'] >= cr_quantiles[0.33]:
          cr_segment = 'MEDIUM'

      li_segment = 'LOW'
      if row['LI'] >= li_quantiles[0.66]:
          li_segment = 'HIGH'
      elif row['LI'] >= li_quantiles[0.33]:
          li_segment = 'MEDIUM'

      lg_segment = 'LOW'
      if row['LG'] >= lg_quantiles[0.66]:
          lg_segment = 'HIGH'
      elif row['LG'] >= lg_quantiles[0.33]:
          lg_segment = 'MEDIUM'

      return f"{cr_segment}-{li_segment}-{lg_segment}"

  # Apply segmentation logic to each user
  df['CR_LI_LG_segment'] = df.apply(segment_user, axis=1)
  return df

# Example usage with pub_usermetrics1
pub_usermetrics1_segmented = create_matrix_segments(pub_usermetrics1)
pub_usermetrics1_segmented = pub_usermetrics1_segmented[['user_id', 'CR_LI_LG_segment']]
pub_usermetrics1_segmented

,user_id,CR_LI_LG_segment
0,2038,LOW-LOW-LOW
1,5746,MEDIUM-LOW-MEDIUM
2,5987,HIGH-LOW-HIGH
3,6005,HIGH-LOW-MEDIUM
4,6021,HIGH-HIGH-LOW
...,...,...
9492,26302,MEDIUM-HIGH-LOW
9493,26305,HIGH-LOW-LOW
9494,26306,HIGH-MEDIUM-LOW
9495,26307,HIGH-MEDIUM-LOW


Reco

In [ ]:
user_df = pub_usermetrics1
question_df = q_total1

# Function to recommend questions with diversification and similarity threshold
def recommend_questions(user_id, topic_name, top_n=no_recommendation, min_similarity=0.2, max_per_learning_unit=50):
    # Candidate Generation by Similarity
    user_row = user_df[user_df['user_id'] == user_id]
    if user_row.empty:
        print("User ID not found.")
        return []

    user_segment = user_row['CR_LI_LG_segment'].values[0]

    # Combine text fields and apply TF-IDF
    question_df['combined_text'] = question_df['learning_unit'] + " " + question_df['Bloom_tag']
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(question_df['combined_text'])

    # Calculate similarity with the topic_name
    topic_vector = tfidf.transform([topic_name])
    cosine_similarities = cosine_similarity(topic_vector, tfidf_matrix).flatten()

    # Apply similarity threshold
    candidate_indices = cosine_similarities.argsort()[::-1]
    candidates = question_df.iloc[candidate_indices].copy()
    candidates['similarity'] = cosine_similarities[candidate_indices]
    candidates = candidates[candidates['similarity'] >= min_similarity]

    # Step 2: Scoring
    def score_question(row):
        segment_score = 0
        if 'HIGH' in user_segment and row['3PL'] < -2:
            segment_score += 3
        elif 'MEDIUM' in user_segment and -2 <= row['3PL'] < 0:
            segment_score += 2
        elif 'LOW' in user_segment and row['3PL'] >= 0:
            segment_score += 1

        total_score = segment_score + row['competency_score'] * 0.5 + row['similarity'] * 2
        return total_score

    candidates['score'] = candidates.apply(score_question, axis=1)

    # Step 3: Reranking and Diversification
    candidates = candidates.sort_values(by='score', ascending=False)
    diversified_recommendations = []

    learning_unit_counts = {}

    for _, row in candidates.iterrows():
        lu = row['learning_unit']

        # Limit recommendations per learning_unit
        if learning_unit_counts.get(lu, 0) < max_per_learning_unit:
            diversified_recommendations.append(row)
            learning_unit_counts[lu] = learning_unit_counts.get(lu, 0) + 1

        # Stop when we reach top_n recommendations
        if len(diversified_recommendations) >= top_n:
            break

    # Convert to DataFrame and return
    recommendations_df = pd.DataFrame(diversified_recommendations)
    recommendations_df = recommendations_df[['question_text', 'learning_unit', '3PL', 'score']]

    return recommendations_df

# Example usage of the function
recommended_questions = recommend_questions(user_id=user_id, topic_name=topic_name)
recommended_questions = recommended_questions.drop_duplicates(subset=['question_text'])

# Add ranking
recommended_questions['rank'] = recommended_questions['score'].rank(ascending=False, method='dense')
recommended_questions


,question_text,learning_unit,3PL,score,rank
6842,What is the missing number in the equation sho...,Associative property for addition of whole num...,-2.041220,4.448317,1.0
6922,What does the commutative property of multipli...,Commutative property for multiplication of rat...,-2.007468,4.237390,2.0
6897,Which of the following statements is correct?,Relation between fractional numbers and ration...,-2.024382,4.022070,3.0
6563,Which of the following is a negative rational ...,Definition of negative rational number,-2.197225,3.911182,4.0
6436,Which of the following is a positive rational ...,Definition of positive rational number,-2.288196,3.864673,5.0
9669,Mahima and Lubna solve division problems invol...,Examination of associative property for divisi...,-1.203973,3.476631,6.0
1876,Which of the given relations demonstrates the ...,Associative property for multiplication of who...,-1.029619,3.367074,7.0
2091,Consider the equation below.55 + (25 + 45) = x...,Associative property for addition of whole num...,-0.916291,3.362931,8.0
2228,Consider the equation below:12 × (13 × 14) = (...,Associative property for multiplication of who...,-0.847298,3.360793,9.0
11460,Which of the following options demonstrates th...,Associative property for addition of whole num...,-0.847298,3.355788,10.0
